In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import genfromtxt
import theano
import theano.gpuarray
import pygpu
from pygpu import gpuarray
theano.gpuarray.use("cuda"+str(0))
from six.moves import cPickle
import os
import matplotlib.colors as mcolors
from mpl_toolkits.axes_grid1 import make_axes_locatable
import statistics as stats

e={0:'EW',1:'EA',2:'RRab',3:'RRc',4:'RRd',5:'RSCVn',6:'LPV'}
g={0:'AGN',1:'SN', 2:'Fl', 3:'CV', 4:'Bl'}

f=open('n_obj.save','rb')
n_net=cPickle.load(f)
f.close()

f=open('p_obj.save','rb')
p_net=cPickle.load(f)
f.close()

def ptf(x):
    
    n=np.zeros(5)
    p=np.zeros(7)

    (q,c,d)=x.shape

    n=n_net.predict_proba(x.reshape(1,1,c,d))[0]
    p=p_net.predict_proba(x.reshape(1,1,c,d))[0]
    
    n_sort=sorted(n)
    p_sort=sorted(p)
    
    n_med=stats.median(n)
    p_med=stats.median(p)
    
    n_mean=n.mean()
    p_mean=p.mean()
    
    n_max=n.max()
    p_max=p.max()
    
    print("Median: "+str(n_med)#+"\nMean: "+str(n_mean)
          +"\nNon-Periodic: "+str(n))
    print("Median: "+str(p_med)#+"\nMean: "+str(p_mean)
          +"\nPeriodic: "+str(p))
    
    print(n_sort[4]-n_sort[3])
    print(p_sort[6]-p_sort[5])
    
    if abs(n_max-p_max)>=0.1:
        if n_max>=0.8 and n_med>p_med:
            r=np.where(n==n_max)[0]
            print(g[r[0]]+", Non-Periodic: "+str(n_max))

        elif p_max>=0.8 and p_med>n_med:
            r=np.where(p==p_max)[0]
            print(e[r[0]]+", Periodic: "+str(p_max))

        elif max(p_max,n_max)<0.3:
            print("None of the above class")
            
    else:
        
        if (p_sort[6]-p_sort[5])>(n_sort[4]-n_sort[3]):
            r=np.where(p==p_max)[0]
            print(e[r[0]]+", Periodic: "+str(p_max))

        if (p_sort[6]-p_sort[5])<(n_sort[4]-n_sort[3]):
            r=np.where(n==n_max)[0]
            print(g[r[0]]+", Non-Periodic: "+str(n_max))
        
        


WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
Can not use cuDNN on context None: cannot compile with cuDNN. We got this error:
b'/tmp/try_flags_18epx42f.c:4:10: fatal error: cudnn.h: No such file or directory\n #include <cudnn.h>\n          ^~~~~~~~~\ncompilation terminated.\n'
Mapped name None to device cuda0: Tesla P100-PCIE-12GB (0000:81:00.0)
/data/mbg/anaconda3/lib/python3.6/site-packages/lasagne/layers/corrmm.py:37: UserWarning: You are using a GPU layer with Theano configured for double precision (floatX=float64). Depending on your Theano version and GPU, this may be slow or unsupported. We recommend to configure Theano for single precision (floatX=float32); see http://lasagne.readthedocs.org/en/latest/user/installation.html#gpu-support.
  warnings.warn("You are using a GPU layer with Theano configured for "
/data/mbg/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in 

In [2]:
x=np.load("../periodic/code/experiments/cnn/X_test.npy")

In [3]:
y=np.load("../periodic/code/experiments/cnn/y_test.npy")

In [4]:
y[343]

1

In [6]:
ptf(x[343])

Median: 8.947831272117256e-08
Non-Periodic: [2.44691310e-11 1.29772225e-05 9.99986933e-01 8.94783127e-08
 1.59190065e-17]
Median: 0.007700671721463592
Periodic: [0.54378112 0.00589847 0.00172602 0.0086509  0.00770067 0.43002284
 0.00221998]
0.9999739560522062
0.1137582726869295
